In [1]:
import numpy
import os
import pandas
import datetime
import matplotlib.pyplot as plt
import world_bank_data as wb

In [2]:
df1 = pandas.read_excel('./COVID-19-geographic-disbtribution-worldwide.xlsx')

In [3]:
aggregation_functions = {
    'cases': 'sum',
    'deaths': 'sum',
    'countryterritoryCode': 'first',
    'popData2019': 'first',
    'continentExp': 'first',
}

df_new = df1.copy()
#df_new.drop(df_new[(df_new.month < 11) | (df_new.year < 2020)].index, inplace=True)
df_new.drop(df_new[df_new.dateRep < '2020-09-01'].index, inplace=True)
df_new = df_new.groupby(['countriesAndTerritories']).aggregate(aggregation_functions)

df_november = df1.copy()
df_november.drop(df_november[df_november.dateRep < '2020-11-01'].index, inplace=True)
df_november = df_november.groupby(['countriesAndTerritories']).aggregate({
    'cases': 'sum',
    'deaths': 'sum'})

In [4]:
# calculate cases / deaths for population
df_new['casesSinceNovember'] = df_november['cases']
df_new['deathsSinceNovember'] = df_november['deaths']
df_new['casesPer1000'] = df_new['cases'] / df_new['popData2019'] * 1000
df_new['deathsPer1000'] = df_new['deaths'] / df_new['popData2019'] * 1000
df_new['NovemberCasesPercent'] = df_new['casesSinceNovember'] / df_new['cases']
df_new['NovemberDeathsPercent'] = df_new['deathsSinceNovember'] / df_new['deaths']

In [5]:
# simple cases/ deaths plot
# df_new[['casesPer1000', 'deathsPer1000']].plot(figsize=(20,10))

In [6]:
# rearange columns
cols = df_new.columns.tolist()
cols = cols[-4:] + cols[:-4]
df_new = df_new[cols]

# rename columns
df_new.rename(columns={"cases": "casesSinceSeptember", "deaths": "deathsSinceSeptember"}, inplace=True)

In [7]:
def get_wbd_by_indicator(indicator: str, mvr_value=20):
    new_wbd_data = pandas.DataFrame(wb.get_series(indicator, mrv=mvr_value, id_or_value='id', simplify_index=True))
    new_wbd_data = new_wbd_data.groupby(['Country']).aggregate({indicator: 'last'})
    return new_wbd_data

# fetch new data from api.world.bank.data
df_gdp_pcap = get_wbd_by_indicator('NY.GDP.PCAP.CD')
df_pop_dnst = get_wbd_by_indicator('EN.POP.DNST')
df_med_beds = get_wbd_by_indicator('SH.MED.BEDS.ZS')
df_med_phys = get_wbd_by_indicator('SH.MED.PHYS.ZS')
df_pop_65up = get_wbd_by_indicator('SP.POP.65UP.TO.ZS')
df_sta_traf = get_wbd_by_indicator('SH.STA.TRAF.P5')
df_atm_pm25 = get_wbd_by_indicator('EN.ATM.PM25.MC.M3')

In [8]:
# join new data to DataFrame
df_new = df_new.join(df_gdp_pcap, how='left', on='countryterritoryCode')
df_new = df_new.join(df_pop_dnst, how='left', on='countryterritoryCode')
df_new = df_new.join(df_med_beds, how='left', on='countryterritoryCode')
df_new = df_new.join(df_med_phys, how='left', on='countryterritoryCode')
df_new = df_new.join(df_pop_65up, how='left', on='countryterritoryCode')
df_new = df_new.join(df_sta_traf, how='left', on='countryterritoryCode')
df_new = df_new.join(df_atm_pm25, how='left', on='countryterritoryCode')

In [9]:
# clear empty data with NaN
df_new['casesPer1000'].replace(0, numpy.nan, inplace=True)
df_new['deathsPer1000'].replace(0, numpy.nan, inplace=True)
df_new['NovemberCasesPercent'].replace(0, numpy.nan, inplace=True)
df_new['NovemberDeathsPercent'].replace(0, numpy.nan, inplace=True)

In [10]:
df_new

,casesPer1000,deathsPer1000,NovemberCasesPercent,NovemberDeathsPercent,casesSinceSeptember,deathsSinceSeptember,countryterritoryCode,popData2019,continentExp,casesSinceNovember,deathsSinceNovember,NY.GDP.PCAP.CD,EN.POP.DNST,SH.MED.BEDS.ZS,SH.MED.PHYS.ZS,SP.POP.65UP.TO.ZS,SH.STA.TRAF.P5,EN.ATM.PM25.MC.M3
countriesAndTerritories,,,,,,,,,,,,,,,,,,
Afghanistan,0.133485,0.005652,0.357424,0.376744,5078,215,AFG,38041757.0,Asia,1815,81,502.115487,56.937760,0.5,0.2782,2.615794,15.1,56.910808
Albania,6.237015,0.115985,0.369630,0.331325,17853,332,ALB,2862427.0,Europe,6599,110,5352.857411,104.612263,2.9,1.2164,14.202631,13.6,18.200603
Algeria,0.526629,0.014819,0.404358,0.286834,22673,638,DZA,43053054.0,Africa,9168,183,3948.343279,17.730075,1.9,1.7193,6.552778,23.8,38.884011
Andorra,60.398808,0.288801,0.230385,NaN,4601,22,AND,76177.0,Europe,1060,0,40886.391165,163.842553,2.5,3.3333,NaN,7.6,10.307621
Angola,0.333194,0.006599,0.279046,0.200000,10604,210,AGO,31825299.0,Africa,2959,42,2973.591160,24.713052,0.8,0.2146,2.199342,23.6,32.388505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis_and_Futuna,NaN,NaN,NaN,NaN,1,0,NaN,NaN,Oceania,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Western_Sahara,NaN,NaN,NaN,NaN,0,0,ESH,582458.0,Africa,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,0.004081,0.001337,0.075630,0.076923,119,39,YEM,29161922.0,Asia,9,3,968.159048,53.977853,0.7,0.5251,2.902141,21.5,50.456007


In [11]:
df_new.to_csv(r'./crunched_data.csv')